In [2]:
import tensorflow as tf
import scipy.io

/home/arvind/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [9]:
a = tf.truncated_normal([16,128,128,3])
sess = tf.Session()
sess.run(tf.initialize_all_variables())
sess.run(tf.shape(a))

Instructions for updating:
Use `tf.global_variables_initializer` instead.


array([ 16, 128, 128,   3], dtype=int32)

In [10]:
b=tf.reshape(a,[16,49152])
sess.run(tf.shape(b))

array([   16, 49152], dtype=int32)

In [11]:
mat = scipy.io.loadmat('/home/arvind/Documents/Data/cars_annos.mat')
annotations = mat['annotations']
class_names = mat['class_names']

In [36]:
print(annotations[0][706][5][0][0])
print(class_names[0][195][0])

9
smart fortwo Convertible 2012


In [38]:
for i in range((len(annotations[0]))):
    print(annotations[0][i][5][0][0])
    print(class_names[0][annotations[0][i][5][0][0]][0])

1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL Sedan 2012
1
Acura RL

19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6 Sedan 2011
19
Audi S6

48
Buick Verano Sedan 2012
48
Buick Verano Sedan 2012
48
Buick Verano Sedan 2012
48
Buick Verano Sedan 2012
48
Buick Verano Sedan 2012
48
Buick Verano Sedan 2012
48
Buick Verano Sedan 2012
48
Buick Verano Sedan 2012
48
Buick Verano Sedan 2012
48
Buick Verano Sedan 2012
48
Buick Verano Sedan 2012
48
Buick Verano Sedan 2012
48
Buick Verano Sedan 2012
48
Buick Verano Sedan 2012
48
Buick Verano Sedan 2012
48
Buick Verano Sedan 2012
48
Buick Verano Sedan 2012
48
Buick Verano Sedan 2012
48
Buick Verano Sedan 2012
48
Buick Verano Sedan 2012
48
Buick Verano Sedan 2012
48
Buick Verano Sedan 2012
48
Buick Verano Sedan 2012
48
Buick Verano Sedan 2012
48
Buick Verano Sedan 2012
48
Buick Verano Sedan 2012
48
Buick Verano Sedan 2012
48
Buick Verano Sedan 2012
48
Buick Verano Sedan 2012
48
Buick Verano Sedan 2012
48
Buick Verano Sedan 2012
48
Buick Verano Sedan 2012
48
Buick Verano Sedan 2012
48
Buick Verano Sedan 2012
48
Buick Verano Sedan 2012
48
Buick Verano Sedan 2012
48
Buick Verano Sedan 2012
4

75
Chrysler Aspen SUV 2009
75
Chrysler Aspen SUV 2009
75
Chrysler Aspen SUV 2009
75
Chrysler Aspen SUV 2009
75
Chrysler Aspen SUV 2009
75
Chrysler Aspen SUV 2009
75
Chrysler Aspen SUV 2009
75
Chrysler Aspen SUV 2009
75
Chrysler Aspen SUV 2009
75
Chrysler Aspen SUV 2009
75
Chrysler Aspen SUV 2009
75
Chrysler Aspen SUV 2009
75
Chrysler Aspen SUV 2009
75
Chrysler Aspen SUV 2009
75
Chrysler Aspen SUV 2009
75
Chrysler Aspen SUV 2009
75
Chrysler Aspen SUV 2009
75
Chrysler Aspen SUV 2009
75
Chrysler Aspen SUV 2009
75
Chrysler Aspen SUV 2009
75
Chrysler Aspen SUV 2009
75
Chrysler Aspen SUV 2009
75
Chrysler Aspen SUV 2009
75
Chrysler Aspen SUV 2009
75
Chrysler Aspen SUV 2009
75
Chrysler Aspen SUV 2009
75
Chrysler Aspen SUV 2009
76
Chrysler Sebring Convertible 2010
76
Chrysler Sebring Convertible 2010
76
Chrysler Sebring Convertible 2010
76
Chrysler Sebring Convertible 2010
76
Chrysler Sebring Convertible 2010
76
Chrysler Sebring Convertible 2010
76
Chrysler Sebring Convertible 2010
76
Chrysler 

100
Ferrari FF Coupe 2012
100
Ferrari FF Coupe 2012
100
Ferrari FF Coupe 2012
100
Ferrari FF Coupe 2012
100
Ferrari FF Coupe 2012
100
Ferrari FF Coupe 2012
100
Ferrari FF Coupe 2012
100
Ferrari FF Coupe 2012
100
Ferrari FF Coupe 2012
100
Ferrari FF Coupe 2012
100
Ferrari FF Coupe 2012
100
Ferrari FF Coupe 2012
100
Ferrari FF Coupe 2012
100
Ferrari FF Coupe 2012
100
Ferrari FF Coupe 2012
100
Ferrari FF Coupe 2012
100
Ferrari FF Coupe 2012
100
Ferrari FF Coupe 2012
100
Ferrari FF Coupe 2012
100
Ferrari FF Coupe 2012
100
Ferrari FF Coupe 2012
100
Ferrari FF Coupe 2012
100
Ferrari FF Coupe 2012
100
Ferrari FF Coupe 2012
100
Ferrari FF Coupe 2012
100
Ferrari FF Coupe 2012
100
Ferrari FF Coupe 2012
100
Ferrari FF Coupe 2012
100
Ferrari FF Coupe 2012
100
Ferrari FF Coupe 2012
100
Ferrari FF Coupe 2012
100
Ferrari FF Coupe 2012
100
Ferrari FF Coupe 2012
100
Ferrari FF Coupe 2012
100
Ferrari FF Coupe 2012
100
Ferrari FF Coupe 2012
100
Ferrari FF Coupe 2012
101
Ferrari California Convertible 201

126
Honda Odyssey Minivan 2007
126
Honda Odyssey Minivan 2007
126
Honda Odyssey Minivan 2007
126
Honda Odyssey Minivan 2007
126
Honda Odyssey Minivan 2007
126
Honda Odyssey Minivan 2007
126
Honda Odyssey Minivan 2007
126
Honda Odyssey Minivan 2007
126
Honda Odyssey Minivan 2007
126
Honda Odyssey Minivan 2007
126
Honda Odyssey Minivan 2007
126
Honda Odyssey Minivan 2007
126
Honda Odyssey Minivan 2007
126
Honda Odyssey Minivan 2007
126
Honda Odyssey Minivan 2007
126
Honda Odyssey Minivan 2007
126
Honda Odyssey Minivan 2007
126
Honda Odyssey Minivan 2007
126
Honda Odyssey Minivan 2007
126
Honda Odyssey Minivan 2007
126
Honda Odyssey Minivan 2007
126
Honda Odyssey Minivan 2007
126
Honda Odyssey Minivan 2007
126
Honda Odyssey Minivan 2007
126
Honda Odyssey Minivan 2007
126
Honda Odyssey Minivan 2007
126
Honda Odyssey Minivan 2007
126
Honda Odyssey Minivan 2007
127
Honda Accord Coupe 2012
127
Honda Accord Coupe 2012
127
Honda Accord Coupe 2012
127
Honda Accord Coupe 2012
127
Honda Accord Cou

Lamborghini Gallardo LP 570-4 Superleggera 2012
151
Lamborghini Gallardo LP 570-4 Superleggera 2012
151
Lamborghini Gallardo LP 570-4 Superleggera 2012
151
Lamborghini Gallardo LP 570-4 Superleggera 2012
151
Lamborghini Gallardo LP 570-4 Superleggera 2012
151
Lamborghini Gallardo LP 570-4 Superleggera 2012
151
Lamborghini Gallardo LP 570-4 Superleggera 2012
151
Lamborghini Gallardo LP 570-4 Superleggera 2012
151
Lamborghini Gallardo LP 570-4 Superleggera 2012
151
Lamborghini Gallardo LP 570-4 Superleggera 2012
151
Lamborghini Gallardo LP 570-4 Superleggera 2012
151
Lamborghini Gallardo LP 570-4 Superleggera 2012
151
Lamborghini Gallardo LP 570-4 Superleggera 2012
151
Lamborghini Gallardo LP 570-4 Superleggera 2012
151
Lamborghini Gallardo LP 570-4 Superleggera 2012
151
Lamborghini Gallardo LP 570-4 Superleggera 2012
151
Lamborghini Gallardo LP 570-4 Superleggera 2012
151
Lamborghini Gallardo LP 570-4 Superleggera 2012
151
Lamborghini Gallardo LP 570-4 Superleggera 2012
151
Lamborghini 

Spyker C8 Convertible 2009
178
Spyker C8 Convertible 2009
178
Spyker C8 Convertible 2009
178
Spyker C8 Convertible 2009
178
Spyker C8 Convertible 2009
178
Spyker C8 Convertible 2009
178
Spyker C8 Convertible 2009
178
Spyker C8 Convertible 2009
178
Spyker C8 Convertible 2009
178
Spyker C8 Convertible 2009
178
Spyker C8 Convertible 2009
178
Spyker C8 Convertible 2009
178
Spyker C8 Convertible 2009
178
Spyker C8 Convertible 2009
178
Spyker C8 Convertible 2009
178
Spyker C8 Convertible 2009
178
Spyker C8 Convertible 2009
178
Spyker C8 Convertible 2009
178
Spyker C8 Convertible 2009
178
Spyker C8 Convertible 2009
178
Spyker C8 Convertible 2009
178
Spyker C8 Convertible 2009
178
Spyker C8 Convertible 2009
178
Spyker C8 Convertible 2009
178
Spyker C8 Convertible 2009
178
Spyker C8 Convertible 2009
178
Spyker C8 Convertible 2009
178
Spyker C8 Convertible 2009
178
Spyker C8 Convertible 2009
178
Spyker C8 Convertible 2009
178
Spyker C8 Convertible 2009
178
Spyker C8 Convertible 2009
178
Spyker C

IndexError: index 196 is out of bounds for axis 0 with size 196